# CW2 #


In [1]:
%matplotlib inline
from agents import *
from notebook import psource
import random


In [2]:
psource(Agent)

The `Agent` has two methods.
* `__init__(self, program=None)`: The constructor defines various attributes of the Agent. These include

    * `alive`: which keeps track of whether the agent is alive or not 
    
    * `bump`: which tracks if the agent collides with an edge of the environment (for eg, a wall in a park)
    
    * `holding`: which is a list containing the `Things` an agent is holding, 
    
    * `performance`: which evaluates the performance metrics of the agent 
    
    * `program`: which is the agent program and maps an agent's percepts to actions in the environment. If no implementation is provided, it defaults to asking the user to provide actions for each percept.
    
* `can_grab(self, thing)`: Is used when an environment contains things that an agent can grab and carry. By default, an agent can carry nothing.

## ENVIRONMENT
Now, let us see how environments are defined. Running the next cell will display an implementation of the abstract `Environment` class.

In [3]:
psource(Environment)


`Environment` class has lot of methods! But most of them are incredibly simple, so let's see the ones we'll be using in this notebook.

* `thing_classes(self)`: Returns a static array of `Thing` sub-classes that determine what things are allowed in the environment and what aren't

* `add_thing(self, thing, location=None)`: Adds a thing to the environment at location

* `run(self, steps)`: Runs an environment with the agent in it for a given number of steps.

* `is_done(self)`: Returns true if the objective of the agent and the environment has been completed

The next two functions must be implemented by each subclasses of `Environment` for the agent to recieve percepts and execute actions 

* `percept(self, agent)`: Given an agent, this method returns a list of percepts that the agent sees at the current time

* `execute_action(self, agent, action)`: The environment reacts to an action performed by a given agent. The changes may result in agent experiencing new percepts or other elements reacting to agent input.

## SIMPLE AGENT AND ENVIRONMENT

Let's begin by using the `Agent` class to creating our first agent - a blind dog.

In [4]:
psource(GraphicEnvironment)

### Q3.1 and Q3.2

A park is an example of an environment because our dog can perceive and act upon it. The <b>Environment</b> class is an abstract class, so we will have to create our own subclass from it before we can use it.

In [5]:
#Defining my objects chair, trolley, and person
class Chair(Thing):
    pass

class Trolley(Thing):
    pass

class Person(Thing):
    pass

    

In [6]:
class Park2D(GraphicEnvironment):
    
    def percept(self, agent):
        '''Returns a list of things that are in my agent location as long as the
           agent is not travelling back to starting point'''
        things = self.list_things_at(agent.location) #Acts like a camera for the robot
        if agent.travellingBack == True:
            return ['Home']
        else:
            return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            personItem= self.list_things_at([agent.location[0],agent.location[1]+1],tclass=Person)
            if len(personItem)!=0:
                agent.divertPath(self)
            else:
                agent.move(action)                                                                   
                print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            
        elif action == "sprayChair":
            items = self.list_things_at(agent.location, tclass=Chair)
            if len(items) != 0:
                if agent.spray(items[0]): #Have the dog eat the first item
                    print('{} sprayed the object {} at location: {}'.format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    agent.travellingBack=True # Tells agent to go back home after spraying A chair
                    agent.homePath = agent.forgePath([0,0],self) # Tells agent to form A path to the starting pos
            
         
            
        elif action == "move up":
            '''If robot camera cannot detect person in the direction it wants to go
               robot is told to continue with path, if the agent finds A class person its
               told to find A new path which could avoid touching person'''
            personItem= self.list_things_at([agent.location[0],agent.location[1]-1],tclass=Person)
            if len(personItem)!=0:
                agent.divertPath(self)
            else:                                                                   
                agent.move(action)
                print('{} decided to {} at location: {}'.format(str(agent)[1:-1],action, agent.location))
                
        elif action == "move left":
            '''If robot camera cannot detect person in the direction it wants to go
               robot is told to continue with path, if the agent finds A class person its
               told to find A new path which could avoid touching person'''
            personItem= self.list_things_at([agent.location[0]-1,agent.location[1]],tclass=Person)
            if len(personItem)!=0:
                agent.divertPath(self)
            else:
                print('{} decided to {} at location: {}'.format(str(agent)[1:-1],action, agent.location))
                agent.move(action)
        elif action == "move right":
            '''If robot camera cannot detect person in the direction it wants to go
               robot is told to continue with path, if the agent finds A class person its
               told to find A new path which could avoid touching person'''
            personItem= self.list_things_at([agent.location[0]+1,agent.location[1]],tclass=Person)
            if len(personItem)!=0:
                agent.divertPath(self)
            else:
                print('{} decided to {} at location: {}'.format(str(agent)[1:-1],action, agent.location))
                agent.move(action)
        elif action == "sprayTrolley":
            print(agent.travellingBack)
      
            items = self.list_things_at(agent.location, tclass=Trolley)
            if len(items) != 0:
                if agent.spray(items[0]): #Have the dog eat the first item
                    print('{} sprayed the object {} at location: {}'
                            .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) 
                    agent.travellingBack=True # Tells agent to go back home after spraying A chair
                    agent.homePath = agent.forgePath([0,0],self) # Tells agent to form A path to the starting pos
                    print(agent.homePath)
           
                
        elif action == "continue home":
            '''Each time it is ran we pop the immediate direction agent must take to get to starting pos
               from the directions table that consists of directions to get to home'''
            if len(agent.homePath)!= 0 and agent.travellingBack==True:
                print("popping")
                next_action = agent.homePath.pop(0)
                if next_action == 'move up':
                    print("moved up")
                    agent.move(next_action)
                elif next_action == 'move down':
                    agent.move(next_action)
                elif next_action == 'move right':
                    agent.move(next_action)
                elif next_action == 'move left':
                    print("moved left")
                    agent.move(next_action)
            elif len(agent.homePath)==0 and agent.travellingBack==True:
                print("set to false")
                agent.homePath = []
                agent.travellingBack=False
                    
                
                
                           
     
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Chair) or isinstance(thing, Trolley) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles
 
    def list_all_obj(self,obj):
        '''Lists all objects in the enviroment returning the object and location'''
        total_things=[]
        for i in range(4):
            rowItems = self.list_things_at([i,0],tclass=obj)
            if len(rowItems)!=0:
                for rows in range(len(rowItems)):
                    total_things.append([rowItems[rows],rowItems[rows].location])
            for j in range(4):
                columnItems = self.list_things_at([i,j],tclass=obj)
                if len(columnItems)!=0:
                     for columns in range(len(columnItems)):
                            total_things.append([columnItems[columns],columnItems[columns].location])
                   
        return total_things
                
       
    def findEmptySpot(self):
        possibleSpots =[]
        for i in range(0,self.width):
            if len(self.list_all_obj(Person))!=0 and (self.list_all_obj(Person)[0][1] == [i,0] or i == 0) :
                continue
            else:
                possibleSpots.append([i,0])
            
            for j in range(0,self.height):
                if len(self.list_all_obj(Person))!=0 and (self.list_all_obj(Person)[0][1] == [i,j] or (i == 0 and j ==0)):
                    continue
                else:
                    possibleSpots.append([i,j])
            possibleSpots.remove([0,0])
        return possibleSpots[random.randint(0,len(possibleSpots)-1)]
            
                
                           
            
                     
class Robot(Agent):
    location = [0,0] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing
    travellingBack=False # variable to notiify enviroment when going back to start pos
    homePath = [] # List that consists of directions to go home
        
    def spray(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Trolley) or isinstance(thing, Chair):
            return True
        return False
    
    def move(self,direction):   
        '''Function in charge of making the agent move anywhere'''
        if direction == 'move down':
            self.moveDown()
        elif direction == 'move up':
            self.moveUp()
        elif direction == 'move left':
            self.moveLeft()
        elif direction == 'move right':
            self.moveRight()
        
        if self.location == [0,0]:
            self.travellingBack=False
            
    def moveDown(self):
        if self.location[1] < 4:
            self.location[1] += 1
    def moveUp(self):
        if self.location[1] > 0:
            self.location[1] -=1
    
    def moveRight(self):
        if self.location[0]  <4:
            self.location[0] += 1
            
    def moveLeft(self):
        if self.location[0] > 0:
            self.location[0] -= 1
    
    def forgePath(self,goalLocation,park):
        '''My own algorithm created to return list of directions to go from the agents current location
            to the goal state in the parameter'''
        '''It works by taking the x and y axis of the goal location and the current location calculating the
        difference in each axis and make appropriate actions to get current location to goal'''
        path = []
        currentLocation=self.location[:]
        if goalLocation[1] > self.location[1]:
            #Go down
            for i in range(goalLocation[1]-self.location[1]):
                path.append('move down')
        elif goalLocation[1] < self.location[1]:       
            #Go up
            for i in range(self.location[1]-goalLocation[1]):
                path.append('move up')
        if goalLocation[0] > self.location[0]:
            #Go right
            for i in range(goalLocation[0]-self.location[0]):
                path.append('move right')
        elif goalLocation[0] < self.location[0]:
            #Go left
            for i in range(self.location[0]-goalLocation[0]):
                path.append('move left')
        
        for x in range(0,len(path)):
            '''In charge of manipulating direction table depending if agent will run into person class
                creating small simulation of if the agent was to take such path'''
            if path[x] == 'move up':
                currentLocation[1]-=1
                upCheck=park.list_things_at([currentLocation[0],currentLocation[1]], tclass=Person)
                if len(upCheck)!=0:
                    if path[len(path)-1] == 'move left':
                        path[x] = 'move left'
                        path.insert(x+1,'move up')
                        path.pop(len(path)-1)
                    elif path[len(path)-1] == 'move right':
                        path[x] = 'move right'
                        path.insert(x+1,'move up')
                        path.pop(len(path)-1)
            elif path[x] =='move down':
                currentLocation[1]+=1
                downCheck=park.list_things_at([currentLocation[0],currentLocation[1]], tclass=Person)
                if len(downCheck)!=0:
                    if path[len(path)-1] == 'move left':
                        path[x] = 'move left'
                        path.insert(x+1,'move down')
                        path.pop(len(path)-1)
                    elif path[len(path)-1] == 'move right':
                        path[x] = 'move right'
                        path.insert(x+1,'move down')
                        path.pop(len(path)-1)
            elif path[x] == 'move left':
                currentLocation[0]-=1
                leftCheck=park.list_things_at([currentLocation[0],currentLocation[1]],tclass=Person)
                if len(leftCheck)!=0:
                    print(path[len(path)-1])
                    if path[len(path)-1] == 'move left':
                        path[x] = 'move down'
                        path.insert(x+1,'move left')
                        path.insert(x+2,'move left')
                        path.insert(x+3,'move up')
                        path.pop(len(path)-1)
                   
            elif path[x]=='move right':
                currentLocation[0]+=1
                rightCheck=park.list_things_at([currentLocation[0],currentLocation[1]],tclass=Person)
                if len(rightCheck)!=0:
                    if path[len(path)-1] == 'move right':
                        path[x] = 'move down'
                        path.insert(x+1,'move right')
                        path.insert(x+2,'move right')
                        path.insert(x+3,'move up')
                        path.pop(len(path)-1)
            
        return path
    
                           
                           
            
                           
            
    def divertPath(self,park):
        #If person has 
        options = []
        upCheck=park.list_things_at([self.location[0],self.location[1]-1], tclass=Person)
        downCheck=park.list_things_at([self.location[0],self.location[1]+1], tclass=Person)
        rightCheck=park.list_things_at([self.location[0]+1,self.location[1]],tclass=Person)
        leftCheck=park.list_things_at([self.location[0]-1,self.location[1]],tclass=Person)
        if len(upCheck)!=0:
            options.append('move down')
            
        if len(downCheck)!=0:
            options.append('move up')
        if len(rightCheck)!=0:
            options.append('move left')
            
        if len(leftCheck)!=0:
            options.append('move right')
       
        choice = options[random.randint(0,len(options)-1)]
        self.move(choice)
       
    
            
        
  
    

Now let's test this new park with our same dog, food and water. We color our dog with a nice red and mark food and water with orange and blue respectively.

In [7]:
def program(percepts):
    for p in percepts:
        if isinstance(p, Chair):
            return 'sprayChair'
        elif isinstance(p,Trolley):
            return 'sprayTrolley'
        elif p == 'Home':
            print("FROM PROGRAM")
            return 'continue home'
        
    return random.choice(['move down','move up','move right','move left'])

In [8]:
park = Park2D(5,5, color={'Robot': (200,0,0), 'Chair': (0, 200, 200), 'Trolley': (230, 115, 40),'Person': (0,200,0)}) # park width is set to 5, and height to 20




robot = Robot(program)
trolley = Trolley()
chair = Chair()
person = Person()
park.add_thing(chair,[4,4])
park.add_thing(trolley,[1,3])
park.add_thing(person,[1,0])
park.add_thing(robot,[0,0])



print("Robot starts at (0,0) facing downwards")
#park.run() #Uncomment this line for code to run

Robot starts at (0,0) facing downwards


# 3.3

In [9]:
class Park2D(GraphicEnvironment):
    
    
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        chairObj= self.list_all_obj(Chair)
        trolleyObj = self.list_all_obj(Trolley)
        things = self.list_things_at(agent.location)
        if  len(chairObj)==0 and len(trolleyObj)==0:
            return ['Home']
            print("percept to go home")
            
        elif agent.Initialised==False:
            print("initiaalising")
            return ['Begin']
        elif len(things)!=0 and agent.Initialised==True and len(agent.objPath)==0  and len(agent.homePath)==0:
            print(agent.objLocations)
            for objs in agent.objLocations:
                if things[0] in objs:
                    return things
                    
        elif len(agent.objPath)!=0:
            return ["continue objpath"]
        else:
            return ["Finished"]
        
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
       
        if action == "sprayChair":
            print(agent.travellingBack)
           
            items = self.list_things_at(agent.location, tclass=Chair)
            if len(items) != 0:
                if agent.spray(items[0]): #Have the dog eat the first item
                        
                    print('{} sprayed the object {} at location: {}'.format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    #agent.travellingBack=True
                    chairObj= self.list_all_obj(Chair)
                    trolleyObj = self.list_all_obj(Trolley)
                    if len(chairObj)!= 0:
                        agent.objPath.append(agent.forgePath(chairObj[0][1],self))
                        print("doing chair")
                    if len(trolleyObj)!= 0:
                        if len(chairObj)==0:
                            agent.objPath.append(agent.forgePath(trolleyObj[0][1],self))
                    else:
                        agent.homePath=agent.forgePath([0,0],self)
                        agent.travellingBack == True
                
                    
                    
            
        elif action == "sprayTrolley":
            items = self.list_things_at(agent.location, tclass=Trolley)
            if len(items) != 0:
                if agent.spray(items[0]): #Have the dog eat the first item
                    print('{} sprayed the object {} at location: {}'
                            .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    #agent.homePath = agent.forgePath([0,0],self)
                    chairObj= self.list_all_obj(Chair)
                    trolleyObj = self.list_all_obj(Trolley)
                    if len(chairObj)!= 0:
                        agent.objPath.append(agent.forgePath(chairObj[0][1],self))
                        print("doing chair")
                    if len(trolleyObj)!= 0:
                        if len(chairObj)==0:
                            agent.objPath.append(agent.forgePath(trolleyObj[0][1],self))
                    else:
                        agent.homePath=agent.forgePath([0,0],self)
                        agent.travellingBack == True
                        
           
                
        elif action == "continue objpath":
            if len(agent.objPath)!= 0 and len(agent.objPath[0])!=0:
                print("popping")
                next_action = agent.objPath[0].pop(0)
                if next_action == 'move up':
                    print("moved up")
                    agent.move(next_action)
                elif next_action == 'move down':
                    agent.move(next_action)
                elif next_action == 'move right':
                    agent.move(next_action)
                elif next_action == 'move left':
                    print("moved left")
                    agent.move(next_action)
            elif len(agent.objPath[0]) == 0:
                agent.objPath.pop(0)
          
        elif action == "Begin":
            '''Puts all objects in enviroment in robot list objLocations and stores path for each obj'''
            chairObj= self.list_all_obj(Chair)
            print(chairObj)
            trolleyObj = self.list_all_obj(Trolley)
            if len(chairObj)!= 0:
                agent.objPath.append(agent.forgePath(chairObj[0][1],self))
                print("doing chair")
                for chairs in range(len(chairObj)):
                    agent.objLocations.append([chairObj[chairs][0],chairObj[chairs][1]])
                    
            if len(trolleyObj)!= 0:
                if len(chairObj)==0:
                    agent.objPath.append(agent.forgePath(trolleyObj[0][1],self))
                print("doing trolley")
                for trolleys in range(len(trolleyObj)):
                    agent.objLocations.append([trolleyObj[trolleys][0],trolleyObj[trolleys][1]])
                    
            print(agent.objPath)
            agent.Initialised=True
        elif action == "continue home":
            if len(agent.homePath)!= 0:
                next_action = agent.homePath.pop(0)
                if next_action == 'move up':
                    print("moved up")
                    agent.move(next_action)
                elif next_action == 'move down':
                    agent.move(next_action)
                elif next_action == 'move right':
                    agent.move(next_action)
                elif next_action == 'move left':
                    print("moved left")
                    agent.move(next_action)
            elif len(agent.homePath)==0:
                print("Program finished.")
                agent.homePath = []
                agent.travellingBack=False
            
        
        
            
                
                           
     
    def is_done(self):
                        
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        return False
      
 
    def list_all_obj(self,obj):
        total_things=[]
        for i in range(5):
            rowItems = self.list_things_at([i,0],tclass=obj)
            if len(rowItems)!=0:
                for rows in range(len(rowItems)):
                    total_things.append([rowItems[rows],rowItems[rows].location])
            for j in range(5):
                columnItems = self.list_things_at([i,j],tclass=obj)
                if len(columnItems)!=0:
                     for columns in range(len(columnItems)):
                            total_things.append([columnItems[columns],columnItems[columns].location])
                   
        return total_things
                
       
    def findEmptySpot(self):
        possibleSpots =[]
        for i in range(0,self.width):
            if len(self.list_all_obj(Person))!=0 and (self.list_all_obj(Person)[0][1] == [i,0] or i == 0) :
                continue
            else:
                possibleSpots.append([i,0])
            
            for j in range(0,self.height):
                if len(self.list_all_obj(Person))!=0 and (self.list_all_obj(Person)[0][1] == [i,j] or (i == 0 and j ==0)):
                    continue
                else:
                    possibleSpots.append([i,j])
        return possibleSpots[random.randint(0,len(possibleSpots)-1)]
            
                
    
 
                           
            
                     
class Robot(Agent):
    location = [0,0] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing
    travellingBack=False
    Initialised = False
    homePath = []
    objPath = []
    objLocations=[]
        
    def spray(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Trolley) or isinstance(thing, Chair):
            return True
        return False
    
    def move(self,direction):          
        if direction == 'move down':
            self.moveDown()
        elif direction == 'move up':
            self.moveUp()
        elif direction == 'move left':
            self.moveLeft()
        elif direction == 'move right':
            self.moveRight()
        
        if self.location == [0,0]:
            print("IM BACK HOME")
            self.travellingBack=False
            
    def moveDown(self):
        if self.location[1] < 4:
            self.location[1] += 1
    def moveUp(self):
        if self.location[1] > 0:
            self.location[1] -=1
    
    def moveRight(self):
        if self.location[0]  <4:
            self.location[0] += 1
            
    def moveLeft(self):
        if self.location[0] > 0:
            self.location[0] -= 1
    
    def forgePath(self,goalLocation,park):
        path = []
        currentLocation=self.location[:]
        if goalLocation[1] > self.location[1]:
            #Go down
            for i in range(goalLocation[1]-self.location[1]):
                path.append('move down')
        elif goalLocation[1] < self.location[1]:       
            #Go up
            for i in range(self.location[1]-goalLocation[1]):
                path.append('move up')
        if goalLocation[0] > self.location[0]:
            #Go right
            for i in range(goalLocation[0]-self.location[0]):
                path.append('move right')
        elif goalLocation[0] < self.location[0]:
            #Go left
            for i in range(self.location[0]-goalLocation[0]):
                path.append('move left')
        
        for x in range(0,len(path)):
            
            if path[x] == 'move up':
                currentLocation[1]-=1
                upCheck=park.list_things_at([currentLocation[0],currentLocation[1]], tclass=Person)
                if len(upCheck)!=0:
                    if path[len(path)-1] == 'move left':
                        path[x] = 'move left'
                        path.insert(x+1,'move up')
                        path.pop(len(path)-1)
                    elif path[len(path)-1] == 'move right':
                        path[x] = 'move right'
                        path.insert(x+1,'move up')
                        path.pop(len(path)-1)
            elif path[x] =='move down':
                currentLocation[1]+=1
                downCheck=park.list_things_at([currentLocation[0],currentLocation[1]], tclass=Person)
                if len(downCheck)!=0:
                    if path[len(path)-1] == 'move left':
                        path[x] = 'move left'
                        path.insert(x+1,'move down')
                        path.pop(len(path)-1)
                    elif path[len(path)-1] == 'move right':
                        path[x] = 'move right'
                        path.insert(x+1,'move down')
                        path.pop(len(path)-1)
            elif path[x] == 'move left':
                currentLocation[0]-=1
                leftCheck=park.list_things_at([currentLocation[0],currentLocation[1]],tclass=Person)
                if len(leftCheck)!=0:
                    print(path[len(path)-1])
                    if path[len(path)-1] == 'move left':
                        path[x] = 'move down'
                        path.insert(x+1,'move left')
                        path.insert(x+2,'move left')
                        path.insert(x+3,'move up')
                        path.pop(len(path)-1)
                   
            elif path[x]=='move right':
                currentLocation[0]+=1
                rightCheck=park.list_things_at([currentLocation[0],currentLocation[1]],tclass=Person)
                if len(rightCheck)!=0:
                    if path[len(path)-1] == 'move right':
                        path[x] = 'move down'
                        path.insert(x+1,'move right')
                        path.insert(x+2,'move right')
                        path.insert(x+3,'move up')
                        path.pop(len(path)-1)
            
        return path
    
                           
                           
            
                           
            
    def divertPath(self,park):
        #If person has 
        options = []
        upCheck=park.list_things_at([self.location[0],self.location[1]-1], tclass=Person)
        downCheck=park.list_things_at([self.location[0],self.location[1]+1], tclass=Person)
        rightCheck=park.list_things_at([self.location[0]+1,self.location[1]],tclass=Person)
        leftCheck=park.list_things_at([self.location[0]-1,self.location[1]],tclass=Person)
        if len(upCheck)!=0:
            options.append('move down')
            
        if len(downCheck)!=0:
            options.append('move up')
        if len(rightCheck)!=0:
            options.append('move left')
            
        if len(leftCheck)!=0:
            options.append('move right')
        #If all checks and no person class near you then add all possible movement and move and use this in execute_action
        #print(options)
        choice = options[random.randint(0,len(options)-1)]
        self.move(choice)
       

In [10]:
def program(percepts):
    for p in percepts:
        if isinstance(p,Chair):
            return 'sprayChair'
        elif isinstance(p,Trolley):
            return 'sprayTrolley'
        elif p == 'continue objpath':
            print("FROM PROGRAM")
            return 'continue objpath'
        elif p == 'Begin':
            return 'Begin'
        elif p == 'Home':
            return 'continue home'
        
        
    

In [11]:
park = Park2D(5,5, color={'Robot': (200,0,0), 'Chair': (0, 200, 200), 'Trolley': (230, 115, 40),'Person': (0,200,0)}) # park width is set to 5, and height to 20

m = 3

robot = Robot(program)
for i in range(m):
    obj = random.choice([Trolley(),Chair()])
    park.add_thing(obj,park.findEmptySpot())
person = Person()

park.add_thing(person,[1,0])
park.add_thing(robot,[0,0])



print("Robot starts at (0,0) facing downwards, lets see if he can find any objects!")
park.run()

,,,,
,,,,
,,,,
,,,,
,,,,


Program finished.


KeyboardInterrupt: 